In [111]:
cd Desktop/Summer\ research/fem/

[Errno 2] No such file or directory: 'Desktop/Summer research/fem/'
/Users/wuyifan/Desktop/Summer research/fem


In [120]:
import numpy as np
import math
import igl
import meshplot as mp
import wildmeshing as wm
import polyfempy as pf

V, F = igl.read_triangle_mesh("data/cube.obj")
#p1 = mp.plot(V, F, shading={"wireframe": True}, return_plot=True)
#p1.add_points(V, shading={"point_color": "blue", "point_size": 0.5});
#p1.save("cylinder")

wm.tetrahedralize("data/cube.obj", "out/out_cube.mesh", mute_log=True, edge_length_r=1/10)
solver = pf.Solver()
solver.load_mesh_from_path("out/out_cube.mesh", vismesh_rel_area=1e-4)
v,f = igl.read_triangle_mesh("out/out_cube.mesh")
print(len(v))

[2020-05-29 11:42:33.968] [polyfem] [info] Loading mesh...
[2020-05-29 11:42:33.969] [geogram] [info] Loading file out/out_cube.mesh...
[2020-05-29 11:42:33.981] [geogram] [info] (FP64) nb_v:834 nb_e:0 nb_f:794 nb_b:0 tri:1 dim:3
[2020-05-29 11:42:33.981] [geogram] [info]  nb_tets:3674
[2020-05-29 11:42:33.981] [geogram] [info] Attributes on vertices: point[3]
[2020-05-29 11:42:34.018] [polyfem] [info] mesh bb min [-0.5, -0.5, -0.5], max [0.5, 0.5, 0.5]
[2020-05-29 11:42:34.018] [polyfem] [info]  took 0.0499647s
834


In [121]:
minn = np.min(V, axis=0)
maxx = np.max(V, axis=0)

def sideset(p):
    if p[1]==minn[1]:
        return 1
    elif p[1]>=maxx[1]-0.01:
        return 2
    return 3

solver.set_boundary_side_set_from_bary(sideset)

"""s: categories by ps"""
ps, ts, s = solver.get_boundary_sidesets()

e1 = ps[s[:,0] == 1, :]
e2 = ps[s[:,0] == 2, :]
e3 = ps[s[:,0] == 3, :]

col = np.zeros_like(s); col[s==2] = 2; col[s==3] = 3
#plot2 = mp.plot(ps, ts, col, return_plot=True)
#plot2.add_points(e2, shading={"point_color": "red", "point_size": 0.05})
#plot2.add_points(e3, shading={"point_color": "blue", "point_size": 0.05})
#plot2.save("col_cube")

In [124]:
settings = pf.Settings()
problem = pf.Problem()

settings.set_pde(pf.PDEs.LinearElasticity)

settings.set_material_params("E", 200)
settings.set_material_params("nu", 0.35)

# set the displacement value for the sideset id
problem.set_displacement(2, [0, 0, 0])
problem.set_force(3, [1.5, 0, 0])

settings.set_problem(problem)

solver.settings(settings)
solver.solve()

p, tri, disp = solver.get_sampled_solution()
m = np.linalg.norm(disp, axis=1)

p_uni, indices, inverse = np.unique(p, return_index=True, return_inverse=True, axis=0)

mises, stress_ten = solver.get_sampled_mises_avg()
mises = mises[indices, :]

arr = solver.get_sampled_mises()
arr = arr[indices, :]
print(len(arr))
for i in range(len(arr)):
    if not arr[i] == 0:
        print(i, arr[i], mises[i])
#print(len(mises))

[2020-05-29 11:47:00.373] [polyfem] [info] simplex_count: 	3674
[2020-05-29 11:47:00.373] [polyfem] [info] regular_count: 	0
[2020-05-29 11:47:00.373] [polyfem] [info] regular_boundary_count: 	0
[2020-05-29 11:47:00.373] [polyfem] [info] simple_singular_count: 	0
[2020-05-29 11:47:00.373] [polyfem] [info] multi_singular_count: 	0
[2020-05-29 11:47:00.373] [polyfem] [info] boundary_count: 	0
[2020-05-29 11:47:00.373] [polyfem] [info] multi_singular_boundary_count: 	0
[2020-05-29 11:47:00.376] [polyfem] [info] non_regular_count: 	0
[2020-05-29 11:47:00.376] [polyfem] [info] non_regular_boundary_count: 	0
[2020-05-29 11:47:00.376] [polyfem] [info] undefined_count: 	0
[2020-05-29 11:47:00.376] [polyfem] [info] total count:	 3674
[2020-05-29 11:47:00.376] [polyfem] [info] Building isoparametric basis...
[2020-05-29 11:47:00.407] [polyfem] [info] Computing polygonal basis...
[2020-05-29 11:47:00.407] [polyfem] [info]  took 1.2773e-05s
[2020-05-29 11:47:00.407] [polyfem] [info] hmin: 0.041879

In [71]:
plot = mp.plot(v=p+disp, f=tri, c=m, return_plot=True)
plot.save("cube_[1.5,0,0]")

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0487340…

Plot saved to file cube_[1.5,0,0].html.


In [72]:
fix = np.zeros((len(p), 1))
for i in range(len(p)):
    pt = p[i]
    if pt[1]>=maxx[1]-0.01:
        fix[i] = 1
    else:
        fix[i] = 0
x = np.hstack((fix,p))
x = np.hstack((x,disp))

In [78]:
doc = open('cube_1.txt','w')
#np.set_printoptions(threshold=np.inf)
for item in x:
    for i in item:
        print(i, file=doc, end = ' ')
    print("", file=doc)
doc.close()